# Genetic Algorithm to solve the D-EVRP-NL
This notebook allows you to work with the implemented library and solve the Dynamic Electric Vehicle Routing Problem with Non-linear charging functions. The main purpose is to give good visualization of the solution and easy handling of network complexity.

## Initialization

First, import required libraries.

In [1]:
# Too work with arguments and script paths
import sys

# scientific libraries and tutilities
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt

# setup pyplot
conf = {'font.size': 20}
plt.rcParams.update(conf)

# GA library
from deap import base
from deap import creator
from deap import tools

# Visualization tools
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot, column
from bokeh.layouts import row as layout_row
from bokeh.models.annotations import Arrow, Label
from bokeh.models.arrow_heads import OpenHead, NormalHead, VeeHead
from bokeh.models import ColumnDataSource, Div, Whisker, Span, Range1d
from bokeh.io import export_svgs, export_png

# Simulation environment tools
sys.path.append('..')
import res.EV_utilities
import res.GA_utilities_1

# Display useful loading of bokeh library and start timer
output_notebook()
t0 = time.time()

Loading BokehJS ...

The following are user-defined variables

In [2]:
# Network size and where are data files stored
networkSize = 70
folderPath = '../data/GA_implementation/'

# Charging stations points
chargingTimePoints = True
timePoints = [0.0, 40.0, 60., 120.]
socPoints = [0.0, 75.0, 90.0, 100.0]

# figure related
FIGURE_WIDTH = 800
FIGURE_HEIGHT = 500

figureFolderPath = "../data/GA_implementation/"+str(networkSize)+"network/"

The following implements some utility functions, not related to main GA

In [3]:
# %% Utilities
def shapeItems(items, shape=(None, 2)):
    numOfItems = len(items)
    columns = shape[1]

    if shape[0] is None:
        rows = int(numOfItems / columns) + int(numOfItems % columns)
    else:
        rows = shape[0]

    gridList = [[None] * columns for _ in range(0, rows)]

    i = 0
    j = 0
    for fig in items:
        if j == columns:
            i += 1
            j = 0
        gridList[i][j] = fig
        j += 1
    return gridList

## Import data
There are three main data files:
    - Time matrix: containing the time it takes to travel along arcs between each node
    - Energy matrix: cantaining the energy it takes to travel along arcs between each node
    - Info file: containing the information of each node

In [4]:
# %% Import time and energy matrices and show a value example

pathTM = folderPath + 'timeMatrix_' + str(networkSize) + 'nodes.csv'
timeMatrix = pd.read_csv(pathTM).set_index("TT(MIN)")

print('Time matrix:\n', timeMatrix, "\n")

pathEM = folderPath + 'energyMatrix_' + str(networkSize) + 'nodes.csv'
energyMatrix = pd.read_csv(pathEM).set_index("ENERGY(AH)")

print('Energy matrix:\n', energyMatrix, '\n')

# Indexing example
print("An index example...")
t = timeMatrix.iat[1, 2]
print("Travel time from 1 to 2: ", t)

e = energyMatrix.iat[1, 2]
print("Energy consumption from 1 to 2: ", e, '\n')


Time matrix:
                  0          1          2          3          4          5  \
TT(MIN)                                                                     
0         0.000000   8.356502  16.123209  14.836564  13.392616   9.867789   
1        14.960005   0.000000  13.535000  10.818365   8.083996  11.737020   
2        18.779469   8.607673   0.000000  19.689762   8.923815  10.009828   
3        19.752689  19.233880   8.754894   0.000000  12.713773  14.788688   
4        11.107851   9.119571  13.099216  15.048761   0.000000  11.073443   
5        13.569591  15.147919  15.652644  11.324876  15.046297   0.000000   
6        17.464498  10.461563  10.940352  11.138421  18.124205  19.835869   
7         8.545763  19.573603  10.955728  13.326397  12.548286   8.809506   
8        19.051816  19.126585  14.470572  10.025858  10.300224  14.314167   
9        19.708899  17.608126  19.607606  18.936274  15.412511   8.099018   
10       19.244528  16.888571   9.415239  17.135746  15.411688

In [5]:
# %% Import nodes information to differentiate among them

infoMatrix = pd.read_csv(folderPath + 'infoMatrix_' + str(networkSize) + 'nodes.csv')

depotDF = infoMatrix[infoMatrix['TYPE'] == 'DEPOT'].dropna(axis=1)
customerDF = infoMatrix[infoMatrix['TYPE'] == 'CUSTOMER'].dropna(axis=1)
csDF = infoMatrix[infoMatrix['TYPE'] == 'CS'].dropna(axis=1)

print('Depots DF:\n', depotDF, '\n')
print('Customers DF:\n', customerDF, '\n')
print('Charge Stations DF\n', csDF, '\n')


Depots DF:
    ID  POS_X  POS_Y   TYPE
0   0    0.0    0.0  DEPOT 

Customers DF:
     ID    DEMAND       POS_X       POS_Y  SERVICE_TIME  TIME_WINDOW_LOW  \
1    1  0.381097   14.813312  -29.273489      1.007486       678.736882   
2    2  0.408219  -97.029553   60.025620      2.248755       680.000000   
3    3  0.053798  -43.318305   69.830098      6.003728       623.356850   
4    4  0.233368   37.132541  133.453625      1.160810       697.072161   
5    5  0.302256  112.712082    0.849134      3.061553       629.976678   
6    6  0.461768  -64.212584 -109.112513      3.871720       600.470041   
7    7  0.487554 -102.013543  -66.565080      3.761231       605.271059   
8    8  0.096205  126.678949   54.553029      9.550936       611.281571   
9    9  0.220654  -37.589036   93.696398      2.562627       644.400828   
10  10  0.494580   64.787774   32.400054      7.623873       610.888516   
11  11  0.110390   31.492086  -36.891120      1.324521       651.483803   
12  12  0.191371 

## Create variables

In [6]:
# Create list with information

networkDict = {'DEPOT_LIST': [], 'CUSTOMER_LIST': [], 'CS_LIST': [],
               'TIME_MATRIX': timeMatrix, 'ENERGY_MATRIX': energyMatrix}

for _, row in depotDF.iterrows():
    networkDict[row['ID']] = res.EV_utilities.DepotNode(row['ID'])
    networkDict['DEPOT_LIST'].append(networkDict[row['ID']])

for _, row in customerDF.iterrows():
    networkDict[row['ID']] = res.EV_utilities.CustomerNode(row['ID'], row['SERVICE_TIME'], row['DEMAND'],
                                                           timeWindowUp=row['TIME_WINDOW_UP'],
                                                           timeWindowDown=row['TIME_WINDOW_LOW'])
    networkDict['CUSTOMER_LIST'].append(networkDict[row['ID']])

figureList = []

for _, row in csDF.iterrows():
    # Simple CS with linear curve
    if chargingTimePoints:
        networkDict[row['ID']] = res.EV_utilities.ChargeStationNode(row['ID'],
                                                                    timePoints=timePoints,
                                                                    socPoints=socPoints)
    else:
        networkDict[row['ID']] = res.EV_utilities.ChargeStationNode(row['ID'])

    networkDict['CS_LIST'].append(networkDict[row['ID']])
    # Append to show as grid
    s = figure(width=400, height=250, title='Charging function of CS ' + str(networkDict[row['ID']].id))

    s.xaxis.axis_label = 'Time (min)'
    s.yaxis.axis_label = 'SOC (%)'
    s.line(networkDict[row['ID']].timePoints, networkDict[row['ID']].socPoints, color='black', line_width=2)
    s.asterisk(networkDict[row['ID']].timePoints, networkDict[row['ID']].socPoints, color='black', size=10)
    
    # Save
    s.output_backend = "svg"
    export_svgs(s, filename=figureFolderPath+"CS"+str(row['ID'])+"_chargingFunction.svg")
    
    figureList.append(s)

# show grid
shapedFigs = shapeItems(figureList, shape=(None, 2))
p = gridplot(shapedFigs, toolbar_location='left')
show(p)


## Assign customers to each EV

In [7]:
# %% Number of cars and their random set to visit. Ensure that each one of them visit at least one

t1 = time.time()

nVehicles = 10
chooseCustomersRandom = True

vehiclesDict = {}

if chooseCustomersRandom:
    # Choose the customers each vehicle will visit randomly
    customersID = [evID for evID in customerDF['ID']]
    nCustomers = len(customersID)
    nCustomersPerCar = [int(nCustomers / nVehicles)] * nVehicles

    if len(customersID) % nVehicles != 0:
        nCustomersPerCar[-1] = int(len(customersID) / nVehicles) + 1

    for i, j in enumerate(nCustomersPerCar):
        print('Car', i, 'must visit', j, 'customer/s')
    print('\n')

    for carId, nCustomersCar in enumerate(nCustomersPerCar):
        carCustomersId = []
        for j in range(0, nCustomersCar):
            index = random.randint(0, len(customersID) - 1)
            carCustomersId.append(customersID.pop(index))
        customersToVisit = [customerId for customerId in carCustomersId]
        print('Car', carId, 'must visit customers with ID:', customersToVisit)

        nodeSequence = [0] + customersToVisit + [0]
        chargingSequence = [0] * len(nodeSequence)

        # instantiate
        Qi = 80.0
        x1 = 24.0 * 30.0
        sumDi = np.sum([networkDict[i].demand for i in nodeSequence])
        vehiclesDict[carId] = res.EV_utilities.ElectricVehicle(carId, customersToVisit, networkDict,
                                                               nodeSequence=nodeSequence,
                                                               chargingSequence=chargingSequence,
                                                               timeMatrix=timeMatrix.iat,
                                                               energyMatrix=energyMatrix.iat,
                                                               x1=x1, x2=Qi, x3=sumDi)
else:
    customersID = [[14,1,2,13,26,11,10,18],
                   [4,25,6,27,9,8,29],
                   [22,16,19,12,17,28],
                   [3,20,21,5,7,23]]
    for i, l in enumerate(customersID):
        print('Car',i,'must visit', len(l),'customers')
        
    nCustomers = sum([len(x) for x in customersID])

    for carId, customersToVisit in enumerate(customersID):
        print('Car', carId, 'must visit customers with ID:', customersToVisit)

        # IMPORTANT: the proposed nodeSequence
        nodeSequence = [0] + customersToVisit + [0]
        chargingSequence = [0] * len(nodeSequence)

        # instantiate
        Qi = 80.0
        sumDi = np.sum([networkDict[i].demand for i in nodeSequence])
        vehiclesDict[carId] = res.EV_utilities.ElectricVehicle(carId, customersToVisit, networkDict,
                                                               nodeSequence=nodeSequence,
                                                               chargingSequence=chargingSequence,
                                                               timeMatrix=timeMatrix.iat,
                                                               energyMatrix=energyMatrix.iat,
                                                               x2=Qi, x3=sumDi)


Car 0 must visit 6 customer/s
Car 1 must visit 6 customer/s
Car 2 must visit 6 customer/s
Car 3 must visit 6 customer/s
Car 4 must visit 6 customer/s
Car 5 must visit 6 customer/s
Car 6 must visit 6 customer/s
Car 7 must visit 6 customer/s
Car 8 must visit 6 customer/s
Car 9 must visit 7 customer/s


Car 0 must visit customers with ID: [64, 9, 31, 33, 5, 47]
Car 1 must visit customers with ID: [16, 25, 40, 34, 15, 63]
Car 2 must visit customers with ID: [30, 59, 61, 44, 43, 53]
Car 3 must visit customers with ID: [35, 36, 49, 2, 55, 50]
Car 4 must visit customers with ID: [26, 32, 39, 67, 42, 18]
Car 5 must visit customers with ID: [19, 20, 21, 56, 46, 8]
Car 6 must visit customers with ID: [10, 4, 1, 27, 57, 58]
Car 7 must visit customers with ID: [11, 66, 45, 29, 28, 14]
Car 8 must visit customers with ID: [12, 22, 7, 3, 54, 62]
Car 9 must visit customers with ID: [38, 6, 37, 17, 24, 48, 60]


 ## Setup GA

In [8]:
# %% Genetic algorithm

# allowed charging operations
numChargeOp = 2

# Build toolbox and useful classes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Individual initializer
toolbox.register("individual", res.GA_utilities_1.createRandomIndividual, vehiclesDict,
                 allowed_charging_operations=numChargeOp)

# Fitness, crossover, mutation and selection
toolbox.register("evaluate", res.GA_utilities_1.fitness, vehiclesDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)
toolbox.register("mate", res.GA_utilities_1.crossover, vehiclesDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)
toolbox.register("mutate", res.GA_utilities_1.mutate, vehiclesDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)
toolbox.register("select", tools.selTournament, tournsize=3)

# Useful to decode
toolbox.register("decode", res.GA_utilities_1.decodeFunction, vehiclesDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)

# Constraint handling
toolbox.register("distance", res.GA_utilities_1.distanceToFeasibleZone, vehicleDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)
toolbox.register("feasible", res.GA_utilities_1.feasibleIndividual, vehicleDict=vehiclesDict,
                 allowed_charging_operations=numChargeOp)
toolbox.decorate("evaluate", tools.DeltaPenality(toolbox.feasible, -500000.0, toolbox.distance))

## Run GA

In [9]:
# %% the algorithm

tInitGA = time.time()
# Population TODO create function
n = 200
generations = 600

pop = []
for i in range(0, n):
    pop.append(creator.Individual(toolbox.individual()))

# CXPB  is the probability with which two individuals
#       are crossed
#
# MUTPB is the probability for mutating an individual
CXPB, MUTPB = 0.4, 0.4

print("################  Start of evolution  ################")

# Evaluate the entire population
# fitnesses = list(map(toolbox.evaluate, pop))

for ind in pop:
    fit = toolbox.evaluate(ind)
    ind.fitness.values = fit

print("  Evaluated %i individuals" % len(pop))

# Extracting all the fitnesses of
fits = [ind.fitness.values[0] for ind in pop]

# Variable keeping track of the number of generations
g = 0
Ymax = []
Ymin = []
Yavg = []
Ystd = []
X = []

bestOfAll = tools.selBest(pop, 1)[0]

# Begin the evolution
while g < generations:
    # A new generation
    g = g + 1
    X.append(g)
    print("-- Generation %i --" % g)

    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):

        # cross two individuals with probability CXPB
        if random.random() < CXPB:
            toolbox.mate(child1, child2)

            # fitness values of the children
            # must be recalculated later
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:

        # mutate an individual with probability MUTPB
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    for ind in invalid_ind:
        fit = toolbox.evaluate(ind)
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(invalid_ind))

    # The population is entirely replaced by the offspring
    pop[:] = offspring

    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5

    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

    Ymax.append(-max(fits))
    Ymin.append(-min(fits))
    Yavg.append(mean)
    Ystd.append(std)

    bestInd = tools.selBest(pop, 1)[0]
    print("Best individual: ", bestInd)

    worstInd = tools.selWorst(pop, 1)[0]
    print("Worst individual: ", worstInd)

    # Save best ind
    if bestInd.fitness.values[0] > bestOfAll.fitness.values[0]:
        bestOfAll = bestInd

# %%
print("################  End of (successful) evolution  ################")


################  Start of evolution  ################
  Evaluated 200 individuals
-- Generation 1 --
  Evaluated 121 individuals
  Min -11253666.930163585
  Max -2725044.485564634
  Avg -6613893.161256376
  Std 1533462.3931694117
Best individual:  [31, 47, 33, 9, 64, 5, -1, 0, 10.0, -1, 0, 10.0, 471, 40, 25, 63, 15, 34, 16, -1, 0, 10.0, -1, 0, 10.0, 436, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 36, 55, 49, 35, 50, -1, 0, 10.0, -1, 0, 10.0, 1030, 42, 32, 18, 39, 67, 26, -1, 0, 10.0, -1, 0, 10.0, 895, 20, 56, 8, 19, 46, 21, -1, 0, 10.0, -1, 0, 10.0, 943, 58, 4, 1, 27, 10, 57, -1, 0, 10.0, -1, 0, 10.0, 545, 45, 29, 14, 28, 11, 66, -1, 0, 10.0, -1, 0, 10.0, 839, 12, 3, 7, 54, 22, 62, -1, 0, 10.0, -1, 0, 10.0, 747, 24, 17, 37, 6, 38, 48, 60, -1, 0, 10.0, -1, 0, 10.0, 296]
Worst individual:  [31, 33, 9, 47, 5, 64, -1, 0, 10.0, -1, 0, 10.0, 1433, 34, 40, 25, 16, 15, 63, -1, 0, 10.0, -1, 0, 10.0, 142, 53, 61, 43, 44, 59, 30, -1, 0, 10.0, -1, 0, 10.0, 1055, 35, 50, 49, 36, 2, 

  Evaluated 121 individuals
  Min -2585485.1196975973
  Max -2033945.7071552526
  Avg -2312035.382745781
  Std 129108.21593869635
Best individual:  [33, 64, 31, 9, 5, 47, -1, 0, 10.0, -1, 0, 10.0, 471, 40, 25, 15, 63, 34, 16, -1, 0, 10.0, -1, 0, 10.0, 633, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 50, 36, 55, 49, 35, 2, -1, 0, 10.0, -1, 0, 10.0, 1030, 42, 32, 18, 39, 67, 26, -1, 0, 10.0, -1, 0, 10.0, 895, 46, 20, 21, 56, 8, 19, -1, 69, 70.23006177275407, -1, 0, 10.0, 904.7881028643241, 58, 4, 10, 27, 1, 57, -1, 0, 10.0, -1, 0, 10.0, 545, 45, 29, 14, 28, 11, 66, -1, 0, 10.0, -1, 0, 10.0, 839, 12, 3, 7, 54, 22, 62, -1, 0, 10.0, -1, 0, 10.0, 747, 24, 17, 6, 37, 38, 48, 60, -1, 0, 10.0, -1, 0, 10.0, 794]
Worst individual:  [31, 47, 33, 9, 64, 5, -1, 0, 10.0, 9, 69, 84.81413351634686, 471, 40, 25, 63, 15, 34, 16, -1, 0, 10.0, -1, 0, 10.0, 436, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 36, 55, 49, 35, 50, -1, 0, 10.0, -1, 0, 10.0, 1030, 42, 32, 18, 39, 67, 26, -1

  Evaluated 143 individuals
  Min -2204384.8945773877
  Max -1869300.3386794177
  Avg -1967534.5199365073
  Std 43277.11058306062
Best individual:  [31, 47, 33, 9, 64, 5, -1, 0, 10.0, -1, 0, 10.0, 520.9746979142161, 34, 25, 63, 15, 40, 16, -1, 0, 10.0, -1, 0, 10.0, 436, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 69, 65.8841741987178, 775, 50, 36, 55, 49, 35, 2, -1, 0, 10.0, -1, 0, 10.0, 982.094387311092, 42, 32, 18, 39, 67, 26, -1, 0, 10.0, -1, 0, 10.0, 895, 20, 56, 8, 19, 46, 21, -1, 0, 10.0, -1, 0, 10.0, 895.5773420476796, 27, 4, 1, 58, 10, 57, -1, 0, 10.0, 4, 69, 46.82472672000208, 545, 45, 29, 14, 11, 28, 66, -1, 0, 10.0, -1, 69, 19.69759665834875, 839, 12, 3, 7, 54, 22, 62, -1, 0, 10.0, -1, 0, 10.0, 747, 24, 17, 37, 6, 38, 48, 60, -1, 0, 10.0, -1, 0, 10.0, 740]
Worst individual:  [33, 64, 31, 9, 5, 47, -1, 0, 10.0, -1, 0, 10.0, 471, 40, 25, 15, 63, 34, 16, -1, 0, 10.0, -1, 0, 10.0, 633, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 49, 55, 36, 35, 50, -1, 0, 10.0, -1, 0,

  Evaluated 136 individuals
  Min -2035406.737239474
  Max -1612350.9911957057
  Avg -1806893.1042959031
  Std 59910.1035350676
Best individual:  [33, 64, 31, 9, 5, 47, -1, 0, 10.0, 31, 68, 46.63623608358696, 471, 40, 25, 15, 16, 34, 63, -1, 0, 10.0, -1, 0, 10.0, 633, 44, 61, 53, 30, 43, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 49, 55, 36, 35, 50, -1, 0, 10.0, -1, 0, 10.0, 982.094387311092, 42, 32, 18, 39, 67, 26, -1, 0, 10.0, -1, 0, 10.0, 851.5898032333155, 20, 56, 8, 19, 46, 21, -1, 69, 70.23006177275407, -1, 0, 10.0, 850.1279644544716, 57, 4, 10, 27, 1, 58, -1, 0, 10.0, -1, 0, 10.0, 545, 45, 29, 14, 28, 11, 66, -1, 0, 10.0, 66, 68, 80.93721158915031, 839, 12, 7, 3, 54, 22, 62, -1, 0, 10.0, -1, 0, 10.0, 747, 24, 17, 6, 37, 38, 48, 60, -1, 0, 10.0, -1, 0, 10.0, 794]
Worst individual:  [31, 64, 33, 9, 47, 5, -1, 0, 10.0, -1, 0, 10.0, 471, 40, 25, 63, 15, 34, 16, -1, 0, 10.0, -1, 0, 10.0, 436, 44, 43, 53, 30, 61, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 49, 55, 36, 35, 50, -1, 0, 10.0, -1, 0,

  Evaluated 132 individuals
  Min -1735658.7324582746
  Max -1535190.0381380874
  Avg -1611394.9278550092
  Std 24185.576753872967
Best individual:  [33, 9, 31, 64, 5, 47, -1, 0, 10.0, 31, 68, 46.63623608358696, 471, 40, 25, 15, 16, 34, 63, -1, 69, 49.99595595306092, -1, 68, 25.951316647790733, 633, 44, 61, 53, 30, 43, 59, -1, 0, 10.0, -1, 0, 10.0, 775, 2, 49, 55, 36, 35, 50, -1, 0, 10.0, -1, 0, 10.0, 982.094387311092, 42, 32, 18, 39, 67, 26, -1, 0, 10.0, -1, 68, 42.979192493153555, 851.5898032333155, 20, 56, 8, 19, 46, 21, -1, 69, 70.23006177275407, -1, 0, 10.0, 850.1279644544716, 57, 4, 10, 27, 1, 58, -1, 0, 10.0, -1, 0, 10.0, 545, 45, 29, 14, 28, 11, 66, -1, 69, 47.29797123598091, 66, 68, 80.93721158915031, 839, 12, 3, 7, 54, 22, 62, -1, 0, 10.0, -1, 0, 10.0, 747, 24, 17, 6, 37, 38, 48, 60, -1, 0, 10.0, -1, 0, 10.0, 744.876168612396]
Worst individual:  [33, 64, 31, 9, 5, 47, -1, 0, 10.0, 31, 68, 46.63623608358696, 471, 40, 25, 15, 16, 34, 63, -1, 68, 13.01148662312674, -1, 0, 10.0, 

...

  Evaluated 117 individuals
  Min -532354.3402149407
  Max -502201.9335046354
  Avg -503774.46076353523
  Std 3371.1806892033856
Best individual:  [31, 5, 64, 9, 33, 47, 5, 69, 4.5153399131626095, -1, 69, 5.149330434949382, 665.3172487076389, 34, 63, 15, 16, 40, 25, -1, 68, 51.04281720607844, 34, 68, 6.847844944938009, 633, 43, 44, 53, 30, 61, 59, -1, 0, 10.0, -1, 68, 50.9462217572822, 664.3983215283988, 35, 2, 55, 50, 49, 36, -1, 68, 86.52322523281752, -1, 68, 21.467367933200975, 719.8850989596513, 42, 32, 39, 18, 67, 26, -1, 0, 10.0, -1, 68, 42.979192493153555, 721.3982269214758, 56, 8, 20, 19, 46, 21, -1, 69, 70.23006177275407, -1, 68, 2.927695616749343, 658.0876302980221, 27, 10, 1, 58, 4, 57, -1, 68, 5.845337618592565, -1, 69, 77.55597563179963, 646.7876919088166, 14, 29, 45, 28, 11, 66, -1, 69, 47.29797123598091, -1, 69, 47.45467964190443, 682.8630486023708, 62, 3, 7, 54, 12, 22, -1, 0, 10.0, -1, 0, 10.0, 659.8356588662052, 48, 37, 6, 38, 17, 24, 60, 24, 68, 15.344135094042615, -

  Evaluated 123 individuals
  Min -522385.8013125055
  Max -501512.65550708043
  Avg -502841.89985916467
  Std 2401.1488360404906
Best individual:  [31, 5, 64, 9, 33, 47, -1, 69, 73.95972765768747, -1, 69, 5.149330434949382, 677.6439638182267, 34, 63, 15, 16, 40, 25, -1, 68, 51.04281720607844, 34, 68, 6.847844944938009, 633, 43, 44, 30, 53, 61, 59, -1, 0, 10.0, -1, 68, 50.9462217572822, 664.3983215283988, 35, 2, 55, 50, 49, 36, -1, 68, 86.52322523281752, -1, 69, 74.85613445855448, 719.8850989596513, 42, 32, 39, 18, 67, 26, -1, 0, 10.0, -1, 68, 42.979192493153555, 675.6918661234312, 56, 8, 20, 19, 46, 21, -1, 68, 39.18864475416043, -1, 69, 25.993566429509848, 658.0876302980221, 27, 10, 4, 58, 1, 57, -1, 68, 5.845337618592565, -1, 69, 77.55597563179963, 646.7876919088166, 14, 29, 45, 28, 11, 66, -1, 69, 47.29797123598091, -1, 69, 47.45467964190443, 682.8630486023708, 62, 3, 7, 54, 12, 22, -1, 0, 10.0, -1, 0, 10.0, 659.8356588662052, 48, 17, 6, 38, 37, 24, 60, 24, 68, 15.344135094042615, 

  Evaluated 139 individuals
  Min -529032.6230494987
  Max -500911.89013221243
  Avg -502044.63507494814
  Std 2500.219700807737
Best individual:  [31, 5, 64, 9, 33, 47, -1, 69, 73.95972765768747, -1, 69, 5.149330434949382, 677.6439638182267, 34, 63, 15, 16, 25, 40, -1, 68, 51.04281720607844, 34, 68, 6.847844944938009, 633, 43, 44, 61, 53, 30, 59, -1, 0, 10.0, -1, 68, 50.9462217572822, 664.3983215283988, 35, 2, 55, 50, 49, 36, -1, 68, 86.52322523281752, -1, 69, 74.85613445855448, 719.8850989596513, 42, 32, 39, 18, 67, 26, -1, 0, 10.0, -1, 68, 42.979192493153555, 675.6918661234312, 46, 8, 56, 19, 20, 21, -1, 68, 39.18864475416043, -1, 69, 25.993566429509848, 658.0876302980221, 27, 10, 4, 58, 1, 57, -1, 68, 5.845337618592565, -1, 69, 77.55597563179963, 646.7876919088166, 14, 29, 45, 28, 11, 66, -1, 69, 47.29797123598091, -1, 69, 47.45467964190443, 682.8630486023708, 62, 3, 7, 54, 12, 22, -1, 0, 10.0, -1, 0, 10.0, 659.8356588662052, 48, 17, 6, 38, 37, 24, 60, 24, 68, 15.344135094042615, -

  Evaluated 128 individuals
  Min -516428.43384118896
  Max -500595.00725835294
  Avg -501441.66069038643
  Std 2148.71599697795
Best individual:  [31, 5, 64, 9, 33, 47, -1, 69, 73.95972765768747, -1, 69, 24.878229465010605, 677.6439638182267, 34, 63, 15, 16, 40, 25, -1, 68, 51.04281720607844, 34, 68, 6.847844944938009, 633, 43, 44, 53, 30, 61, 59, -1, 0, 10.0, -1, 68, 50.9462217572822, 650.4144723504687, 35, 2, 55, 50, 49, 36, -1, 68, 86.52322523281752, -1, 69, 74.85613445855448, 719.8850989596513, 42, 32, 39, 18, 67, 26, -1, 0, 10.0, -1, 68, 42.979192493153555, 675.6918661234312, 56, 8, 20, 19, 46, 21, -1, 68, 39.18864475416043, -1, 69, 25.993566429509848, 658.0876302980221, 27, 10, 4, 58, 1, 57, -1, 68, 5.845337618592565, -1, 69, 77.55597563179963, 646.7876919088166, 14, 29, 45, 28, 11, 66, -1, 69, 47.29797123598091, -1, 69, 47.45467964190443, 682.8630486023708, 7, 3, 62, 54, 12, 22, -1, 0, 10.0, -1, 0, 10.0, 659.8356588662052, 48, 38, 6, 17, 37, 24, 60, 24, 68, 15.344135094042615, 

KeyboardInterrupt: 

### Obtain best individual

In [ ]:
# %% Evaluate the best and decode
bestInd = bestOfAll
toolbox.evaluate(bestInd)
S, L, x0 = toolbox.decode(bestInd)

print('Best individual:\n', bestInd)
print('Best individual fitness:\n', bestInd.fitness)

print('S:\n', S)
print('L:\n', L)
print('x0:\n', x0, '\n')

# Costs
totalTravelTime = 0.0
totalChargingTime = 0.0

leavingReachingSequences = {}

for vehicleId, vehicle in vehiclesDict.items():
    seqEta = np.zeros(vehicle.si)
    leavingReachingSequences[vehicleId] = vehicle.createReachingLeavingStates(seqEta)
    
    reachingTimes = leavingReachingSequences[vehicleId][0]
    leavingTimes = leavingReachingSequences[vehicleId][1]
    spentTimes = leavingTimes - reachingTimes
    
    travelTime = 0.0
    chargingTime = 0.0
    prevLeavingTime = leavingTimes[0]
    
    for node, reachTime, leaveTime, spentTime in zip(S[vehicleId][1:], reachingTimes[1:], leavingTimes[1:], spentTimes[1:]):
        if networkDict[node].isCustomer():
            travelTime += reachTime - prevLeavingTime
        elif networkDict[node].isChargeStation():
            travelTime += reachTime - prevLeavingTime
            chargingTime += spentTime
        else:
            travelTime += reachTime - prevLeavingTime
        prevLeavingTime = leaveTime
            
    print('Travel time vehicle', vehicleId, ':', travelTime)
    print('Charging time vehicle', vehicleId, ':', chargingTime)
    print('Travel + charging times', vehicleId, ':', chargingTime + travelTime, '\n')
    totalTravelTime += travelTime
    totalChargingTime += chargingTime

print('Total travel time:', totalTravelTime)
print('Total charging time:', totalChargingTime)
print('Total:', totalChargingTime + totalTravelTime)

# Execution time
print("GA execution time:", (time.time() - tInitGA) * 1000.0, "ms")

### Theta dynamics

In [ ]:
# %% Obtain CS capacities
sumSi = sum([len(x) for _, x in S.items()])
lenK0 = 2 * sumSi - 2 * nVehicles

leavingReachingSequences = {}

for vehicleId, vehicle in vehiclesDict.items():
    seqEta = np.zeros(vehicle.si)
    leavingReachingSequences[vehicleId] = vehicle.createReachingLeavingStates(seqEta)

theta = np.zeros(networkSize)
theta[0] = nVehicles

aX = []
for vehicleId, leavingReachingMatrix in leavingReachingSequences.items():
    x = []
    for k, nodeId in enumerate(S[vehicleId]):
        # agregar ts, nodo, tipo, vehiculo
        if k == 0:
            # vehicle leaves depot
            x.append([leavingReachingMatrix[1][k], nodeId, -1, vehicleId])
        elif k == len(S[vehicleId]) - 1:
            # vehicle reaches depot
            x.append([leavingReachingMatrix[0][k], nodeId, 1, vehicleId])
        else:
            # vehicle reaches and leaves the k-th node
            x.append([leavingReachingMatrix[0][k], nodeId, 1, vehicleId])
            x.append([leavingReachingMatrix[1][k], nodeId, -1, vehicleId])
    aX.append(x)

# Order
oX = []
for i in range(0, lenK0):
    c = float("inf")
    v = 0
    for j in range(0, nVehicles):
        if not aX[j]:
            pass
        elif aX[j][0][0] < c:
            c = aX[j][0][0]
            v = j
    popEvent = aX[v].pop(0)  # notice that aX will be empty after this. rerun to obtain new results
    oX.append(popEvent)

# Use dynamics

thetaMatrix = np.zeros((networkSize, lenK0 + 1))  # TODO verify this + 1, because of initial conditions
thetaMatrix[:, 0] = theta

for i in range(0, lenK0):
    g, _ = res.EV_utilities.gamma(oX, i, networkSize)
    thetaMatrix[:, i + 1] = thetaMatrix[:, i] + g

## Plot

In [ ]:
# %% Using bokeh

# Statistics

# Fitness per generation
figFitness = figure(plot_width=400, plot_height=300,
                    title='Best fitness evolution')
figFitness.circle(X, np.log(Ymax))
figFitness.xaxis.axis_label = 'Generation'
figFitness.yaxis.axis_label = 'log(-fitness)'

figFitness.output_backend = "svg"
export_svgs(figFitness, filename=figureFolderPath+"fitness.svg")



# Standard deviation of fitness per generation
figFitnessStd = figure(plot_width=400, plot_height=300,
                       title='Standard deviation of best fitness per generation')
figFitnessStd.circle(X, Ystd)
figFitnessStd.xaxis.axis_label = 'Generation'
figFitnessStd.yaxis.axis_label = 'Standard deviation of fitness'
figFitnessStd.left[0].formatter.use_scientific = False

figFitnessStd.output_backend = "svg"
export_svgs(figFitnessStd, filename=figureFolderPath+"fitness_std.svg")

# Grid
p = gridplot([[figFitness, figFitnessStd]], toolbar_location='right')
show(p)


In [ ]:
# %% Vehicles dynamics

# Vectors to plot

colorArrowTravel = 'SteelBlue'
colorArrowCharging = 'Crimson'
colorArrowServing = 'SeaGreen'

# Plot
maxTw = -1
minTw = 100000000000
for vehicleID, vehicle in vehiclesDict.items():
    # figures
    figX1 = figure(plot_width=600, plot_height=450,
                   title='Time the (vehicle '+str(vehicleID)+')',
                   toolbar_location=None)
    figX2 = figure(plot_width=600, plot_height=450,
                   title='SOC (vehicle '+str(vehicleID)+')',
                  y_range=(0, 100),
                  toolbar_location=None)
    figX3 = figure(plot_width=600, plot_height=450,
                   title='Payload (vehicle '+str(vehicleID)+')',
                  toolbar_location=None)
    
    # reaching leaving states for the vehicle
    reachingLeavingStates = leavingReachingSequences[vehicleID]

    # time windows
    nSeq = vehicle.nodeSequence
    kCustomers = []
    tWindowsUpper = []
    tWindowsLower = []
    for i, node in enumerate(nSeq):
        if networkDict[node].isCustomer():
            kCustomers.append(i)
            tWindowsCenter = (networkDict[node].timeWindowUp + networkDict[node].timeWindowDown) / 2.0
            tWindowsWidth =(networkDict[node].timeWindowUp - networkDict[node].timeWindowDown) / 2.0
            tWindowsUpper.append(tWindowsCenter + tWindowsWidth)
            tWindowsLower.append(tWindowsCenter - tWindowsWidth)
            # Time windows whiskers
            whiskerTW = Whisker(base=i, upper=tWindowsCenter + tWindowsWidth, lower=tWindowsCenter - tWindowsWidth)
            figX1.add_layout(whiskerTW)
            
            # update TW bounds
            if tWindowsCenter + tWindowsWidth > maxTw:
                maxTw = tWindowsCenter + tWindowsWidth
            if tWindowsCenter - tWindowsWidth < minTw:
                minTw = tWindowsCenter - tWindowsWidth
    
    # adjust fig 1 to fit TWs
    figX1.y_range=Range1d(minTw-10, maxTw+10)

    kVehicle = range(0, vehicle.si)

    figX1.line(kVehicle, reachingLeavingStates[0, :], alpha=0)
    figX1.line(kVehicle, reachingLeavingStates[1, :], alpha=0)
    figX2.line(kVehicle, reachingLeavingStates[2, :], alpha=0)
    figX2.line(kVehicle, reachingLeavingStates[3, :], alpha=0)
    figX3.line(kVehicle, reachingLeavingStates[4, :], alpha=0)

    reachingLeavingVectorPrev = reachingLeavingStates[:, 0]
    nodePrev = nSeq[0]
    kPrev = 0

    label = Label(x=kPrev, y=reachingLeavingVectorPrev[0], y_offset=-5, text=str(node), text_baseline='top')
    figX1.add_layout(label)

    label = Label(x=kPrev, y=reachingLeavingVectorPrev[2], y_offset=-5, text=str(node), text_baseline='top')
    figX2.add_layout(label)

    label = Label(x=kPrev, y=reachingLeavingVectorPrev[4], y_offset=-5, text=str(node), text_baseline='top')
    figX3.add_layout(label)
    
    # Axes
    figX1.xaxis.axis_label = 'k'
    figX1.yaxis.axis_label = 'Time of the day (min)'
    figX1.axis.axis_label_text_font_size = '15pt'
    figX1.axis.major_label_text_font_size = '13pt'
    figX1.title.text_font_size = '15pt'
    
    figX2.xaxis.axis_label = 'k'
    figX2.yaxis.axis_label = 'SOC (%)'
    figX2.axis.axis_label_text_font_size = '15pt'
    figX2.axis.major_label_text_font_size = '13pt'
    figX2.title.text_font_size = '15pt'
    
    figX3.xaxis.axis_label = 'k'
    figX3.yaxis.axis_label = 'Payload (ton)'
    figX3.axis.axis_label_text_font_size = '15pt'
    figX3.axis.major_label_text_font_size = '13pt'
    figX3.title.text_font_size = '15pt'
    
    # horizontal line SOC
    hline1 = Span(location=40, dimension='width', line_color='black')
    hline2 = Span(location=80, dimension='width', line_color='black')
    figX2.renderers.extend([hline1, hline2])
    
    
    for reachingLeavingVector, node, k in zip(reachingLeavingStates[:, 1:].T, nSeq[1:], range(1, vehicle.si)):
        # x1
        label = Label(x=k, y=reachingLeavingVector[0], y_offset=-5, text=str(node), text_baseline='top')
        figX1.add_layout(label)

        arrowTravel = Arrow(x_start=kPrev, y_start=reachingLeavingVectorPrev[1],
                            x_end=k, y_end=reachingLeavingVector[0],
                            end=VeeHead(size=8, fill_color=colorArrowTravel, line_color=colorArrowTravel),
                            line_color=colorArrowTravel, line_alpha=1)

        if networkDict[node].isChargeStation():
            colorArrowSpent = colorArrowCharging
        else:
            colorArrowSpent = colorArrowServing

        arrowSpent = Arrow(x_start=k, y_start=reachingLeavingVector[0],
                           x_end=k, y_end=reachingLeavingVector[1],
                           end=VeeHead(size=8, fill_color=colorArrowSpent, line_color=colorArrowSpent),
                           line_color=colorArrowSpent, line_alpha=1)

        figX1.add_layout(arrowTravel)
        figX1.add_layout(arrowSpent)

        # x2
        label = Label(x=k, y=reachingLeavingVector[3], y_offset=10, text=str(node), text_baseline='top')
        figX2.add_layout(label)

        arrowTravel = Arrow(x_start=kPrev, y_start=reachingLeavingVectorPrev[3],
                            x_end=k, y_end=reachingLeavingVector[2],
                            end=VeeHead(size=8, fill_color=colorArrowTravel, line_color=colorArrowTravel),
                            line_color=colorArrowTravel, line_alpha=1, line_width=1.5, visible=True)
        figX2.add_layout(arrowTravel)

        if networkDict[node].isChargeStation():
            colorArrowSpent = colorArrowCharging
            arrowSpent = Arrow(x_start=k, y_start=reachingLeavingVector[2],
                               x_end=k, y_end=reachingLeavingVector[3],
                               end=VeeHead(size=8, fill_color=colorArrowSpent, line_color=colorArrowSpent),
                               line_color=colorArrowSpent, line_alpha=1, line_width=1.5, visible=True)
            figX2.add_layout(arrowSpent)


        # x3
        label = Label(x=k, y=reachingLeavingVector[4], y_offset=-5, text=str(node), text_baseline='top')
        figX3.add_layout(label)

        if networkDict[nodePrev].isChargeStation():
            colorArrow = colorArrowCharging

        else:
            colorArrow = colorArrowTravel

        arrowTravel = Arrow(x_start=kPrev, y_start=reachingLeavingVectorPrev[4],
                            x_end=k, y_end=reachingLeavingVector[4],
                            end=VeeHead(size=8, fill_color=colorArrow, line_color=colorArrow),
                            line_color=colorArrow, line_alpha=1, line_width=1.5, visible=True)
        figX3.add_layout(arrowTravel)

        # common
        reachingLeavingVectorPrev = reachingLeavingVector
        nodePrev = node
        kPrev = k
        
    # Save
    export_png(figX1, filename=figureFolderPath+"x1_vehicle"+str(vehicleID)+".png")
    
    export_png(figX2, filename=figureFolderPath+"x2_vehicle"+str(vehicleID)+".png")
    
    export_png(figX3, filename=figureFolderPath+"x3_vehicle"+str(vehicleID)+".png")   
    
    
    # Show
    show(figX1)
    show(figX2)
    show(figX3)

# %% Plot results for CSs
for cs in networkDict['CS_LIST']:
    fig = plt.figure(figsize=(8, 6))
    csID = cs.id
    counter = thetaMatrix[csID, :]
    plt.step(np.arange(len(counter)), counter)
    plt.title('Charging operations at CS ' + str(csID))
    plt.xlabel(r'$\bar{k}$')
    plt.ylabel('Vehicle count')
    plt.savefig(figureFolderPath+"CS_"+str(cs.id)+".png")
    plt.show()


In [ ]:
# %% Time of execution
tEnd = time.time()
print("Total execution time:", (tEnd - t0) * 1000.0, "ms")